## Here lets try to get some data from the alfven modes and train a few autoencoders on them to see if they capture any structure in the latent layer

In [ ]:
using AlfvenDetectors
using PyPlot
using Flux
using CuArrays  # for GPU runs
using ValueHistories
using BSON: @save, @load
using HDF5

What we are doing is unsupervised training on columns of the magnitude squared coherence time histograms. 

### Collect the data

Use shot #10370 and #11960 and several coil couples. Select only some timeslices, normalize data

In [ ]:
host = gethostname()
if occursin("vit", host)
    datapath = "/home/vit/vyzkum/alfven/cdb_data/original_data/"
else
    datapath = "/home/skvara/work/alfven/cdb_data/original_data/"
end

In [ ]:
function get_msc_array(datapath, shot, coil, timelim = [1.0, 1.25])
    h5data = h5open(joinpath(datapath,"$(shot).h5"), "r")
    t = Float32.(read(h5data, "t"))
    msc = Float32.(read(h5data, "cxy$coil"))
    f = Float32.(read(h5data, "f"))    
    tinds = timelim[1] .<= t .<= timelim[2]
    return msc[:,tinds], t[tinds], f 
end

In [ ]:
msc, t, f = get_msc_array(datapath, 11096, 18)

In [ ]:
pcolormesh(t,f,msc)

In [ ]:
function collect_msc(datapath, shot, coils)
    datalist = map(x-> get_msc_array(datapath, shot, x), coils)
    return hcat([x[1] for x in datalist]...), datalist[1][3]
end

In [ ]:
shots_coils = [
#    [10370, [12, 15, 17, 20]],
    [10370, [12, 20]],
#    [11096, [11, 8, 17, 20]]
    [11096, [11, 8, 20]]
]
datalist = map(x->collect_msc(datapath, x[1], x[2]), shots_coils)
data, f = hcat([x[1] for x in datalist]...), datalist[1][2]

In [ ]:
pcolormesh(1:size(data,2), f, data)

### Now that we have the data, construct a TSVAE

Larger dimension of middle layer is beneficial, but improvement from 10 to 20 is much alrger than from 20 to 200.

Reconstruction works even with zdim = 2 although there are some artifacts.

## Basic training seems to work, now test the GPU version

In [ ]:
# convert to CuArrays
zdim = 2
N = size(data,1)
verb = false
cudata = data |> gpu
model = AlfvenDetectors.TSVAE(N, zdim, 2) |> gpu
train_history = (MVHistory(), MVHistory())
nepochs = 100
batchsize = Int(size(data,2)/5)
cbit = 1

In [ ]:
@info "Training a large GPU model with less epochs in more iterations"
# clear cache
for i in 1:5
    @time AlfvenDetectors.fit!(model, cudata, batchsize, nepochs;
        cbit = cbit, history = train_history, verb = verb)
    # clear cache
    GC.gc()
end

In [ ]:
@info "GPU model(data) timing"
@time model(cudata);

In [ ]:
figure()
plot(get(train_history[1], :loss)...,label = "loss")
plot(get(train_history[1], :loglikelihood)..., label="-loglikelihood")
plot(get(train_history[1], :KL)..., label="KL")
title("Training loss of model 1")
xlabel("iteration")
ylabel("loss")
legend()

In [ ]:
figure()
plot(get(train_history[2], :loss)...,label = "loss")
plot(get(train_history[2], :loglikelihood)..., label="-loglikelihood")
plot(get(train_history[2], :KL)..., label="KL")
title("Training loss of model 2")
xlabel("iteration")
ylabel("loss")
legend()

In [ ]:
cpumodel = model |> cpu
figure()
rdata = cpumodel(data).data

pcolormesh(AlfvenDetectors.mu_scalarvar(rdata))
title("TSVAE output with GPU training - mean")
xlabel("t")
ylabel("f")

figure()
pcolormesh(AlfvenDetectors.samplenormal_scalarvar(rdata))
title("TSVAE output with GPU training - sampled")
xlabel("t")
ylabel("f")

figure()
gendata = AlfvenDetectors.sample(cpumodel,100).data
pcolormesh(gendata)
title("TSVAE output with GPU training - generated")
xlabel("t")
ylabel("f")


Check further memory allocation for GPUs

## In this part, lets try to see some sort of structure in the latent code

In [ ]:
# save/load a pretrained model
cpumodel = model |> cpu
f = "tsvae_model.bson"
if !isfile(f) 
    @save f cpumodel
else
    @load f cpumodel
end

In [ ]:
X1, t1, f1 = get_msc_array(datapath, 11096, 11)
pcolormesh(t1, f1, X1)

In [ ]:
X0, t0, f0 = get_msc_array(datapath, 11096, 20)
pcolormesh(t0, f0, X0)

In [ ]:
Xα = X1[:,1.06.<=t1.<=1.22]
zα = AlfvenDetectors.samplenormal(cpumodel.m1.encoder(Xα).data)
z1 = AlfvenDetectors.samplenormal(cpumodel.m1.encoder(X1).data)
z0 = AlfvenDetectors.samplenormal(cpumodel.m1.encoder(X0).data)

In [ ]:
figure()
scatter(z1[1,:], z1[2,:], label = "positive")
scatter(z0[1,:], z0[2,:], label = "negative")
scatter(zα[1,:], zα[2,:], label = "alfven mode")
title("latent space of model 1")
legend()

Also, lets checkout the latent code in the second model.

In [ ]:
uα = cpumodel.m2.encoder(zα).data
u1 = cpumodel.m2.encoder(z1).data
u0 = cpumodel.m2.encoder(z0).data

In [ ]:
figure()
scatter(u1[1,:], u1[2,:], label = "positive")
scatter(u0[1,:], u0[2,:], label = "negative")
scatter(uα[1,:], uα[2,:], label = "alfven mode")
title("latent space of model 2 - means")
legend()

su1 = AlfvenDetectors.samplenormal(u1)
su0 = AlfvenDetectors.samplenormal(u0)
suα = AlfvenDetectors.samplenormal(uα)
figure()
scatter(su1[1,:], su1[2,:], label = "positive")
scatter(su0[1,:], su0[2,:], label = "negative")
scatter(suα[1,:], suα[2,:], label = "alfven mode")
legend()
title("latent space of model 2 - sampled")

